# Library Import

In [1]:
import os
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

In [2]:
import torch

In [3]:
import torch.nn as nn

In [4]:
import torch.nn.functional as F

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
from torch.utils.data import  TensorDataset, DataLoader


# Data Loadd & Preprocessing

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [8]:
path_fault1_ex = 'C:/Users/baek/Desktop/NURO/data/fault1/external'
path_fault1_in = 'C:/Users/baek/Desktop/NURO/data/fault1/internal'

path_normal_ex = 'C:/Users/baek/Desktop/NURO/data/normal/external'
path_normal_in = 'C:/Users/baek/Desktop/NURO/data/normal/internal'

In [9]:
dat_f1 = [[] for i in range(0,2)]
dat_f2 = [[] for i in range(0,2)]
dat_n = []

In [10]:
temp =pd.read_csv(path_fault1_ex + '/'+ os.listdir(path_fault1_ex)[0], sep = '\t' )
dat_f1[0].append(temp)
temp =pd.read_csv(path_fault1_ex + '/'+os.listdir(path_fault1_ex)[1], sep = '\t' )
dat_f1[0].append(temp)
temp =pd.read_csv(path_fault1_ex + '/'+os.listdir(path_fault1_ex)[2], sep = '\t' )
dat_f1[0].append(temp)
temp =pd.read_csv(path_fault1_ex + '/'+os.listdir(path_fault1_ex)[3], sep = '\t' )
dat_f1[0].append(temp)


temp =pd.read_csv(path_fault1_in + '/'+os.listdir(path_fault1_in)[0], sep = ',' , low_memory=False)
dat_f1[1].append(temp)
temp =pd.read_csv(path_fault1_in + '/'+os.listdir(path_fault1_in)[1], sep = ',' , low_memory=False)
dat_f1[1].append(temp)
temp =pd.read_csv(path_fault1_in + '/'+os.listdir(path_fault1_in)[2], sep = ',' , low_memory=False)
dat_f1[1].append(temp)
temp =pd.read_csv(path_fault1_in + '/'+os.listdir(path_fault1_in)[3], sep = ',' , low_memory=False)
dat_f1[1].append(temp)

temp =pd.read_csv(path_normal_ex + '/'+os.listdir(path_normal_ex)[0], sep = '\t' )
dat_n.append(temp)
temp =pd.read_csv(path_normal_ex + '/'+os.listdir(path_normal_ex)[1], sep = '\t' )
dat_n.append(temp)
temp =pd.read_csv(path_normal_ex + '/'+os.listdir(path_normal_ex)[2], sep = '\t' )
dat_n.append(temp)
temp =pd.read_csv(path_normal_ex + '/'+os.listdir(path_normal_ex)[3], sep = '\t' )
dat_n.append(temp)

In [11]:
v1_n = dat_f1[1][0].loc[dat_f1[1][0]['CoreSN'] == 'core0'+str(1),:].reset_index().drop(labels = 'index', axis=1)
v1_m = dat_f1[1][0].loc[dat_f1[1][0]['CoreSN'] == 'core0'+str(2),:].reset_index().drop(labels = 'index', axis=1)

v15_n = dat_f1[1][1].loc[dat_f1[1][1]['CoreSN'] == 'core0'+str(1),:].reset_index().drop(labels = 'index', axis=1)
v15_m = dat_f1[1][1].loc[dat_f1[1][1]['CoreSN'] == 'core0'+str(2),:].reset_index().drop(labels = 'index', axis=1)

v2_n = dat_f1[1][2].loc[dat_f1[1][2]['CoreSN'] == 'core0'+str(1),:].reset_index().drop(labels = 'index', axis=1)
v2_m = dat_f1[1][2].loc[dat_f1[1][2]['CoreSN'] == 'core0'+str(2),:].reset_index().drop(labels = 'index', axis=1)

v25_n = dat_f1[1][3].loc[dat_f1[1][3]['CoreSN'] == 'core0'+str(1),:].reset_index().drop(labels = 'index', axis=1)
v25_m = dat_f1[1][3].loc[dat_f1[1][3]['CoreSN'] == 'core0'+str(2),:].reset_index().drop(labels = 'index', axis=1)

In [12]:
v_n = []
v_m = []

v_n.append(v1_n)
v_n.append(v15_n)
v_n.append(v2_n)
v_n.append(v25_n)

v_m.append(v1_m)
v_m.append(v15_m)
v_m.append(v2_m)
v_m.append(v25_m)

In [13]:
data_n = []
data_m = []

for i in range(0,4):
    for j in range(len(v_n[i])//300):
        data_n.append(v_n[i].loc[300*j:300*(j+1),"ActualTorque"])

for i in range(0,4):
    for j in range(len(v_m[i])//300):
        data_m.append(v_m[i].loc[300*j:300*(j+1),"ActualTorque"])
    
label_n = [0 for i in range(len(data_n))]
label_m = [1 for i in range(len(data_m))]

data_n.extend(data_m)
label_n.extend(label_m)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(data_n,label_n,test_size=0.2, random_state=42)

for i in range(len(X_train)):
    X_train[i] = np.array(X_train[i])
    
for i in range(len(X_test)):
    X_test[i] = np.array(X_test[i])
    
X_test = np.array(X_test)
X_train = np.array(X_train)
y_test = np.array(y_test)
y_train = np.array(y_train)

X_test = X_test.reshape(289,1,301)
X_train = X_train.reshape(1153,1,301)


In [121]:
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)

X_train = X_train.view([-1,1, 301])
X_test = X_test.view([-1,1, 301])

y_train = torch.FloatTensor(y_train)
y_test = torch.FloatTensor(y_test)

y_train  = y_train.view([-1,1])
y_test =  y_test.view([-1,1])

In [126]:
Train = TensorDataset(X_train,y_train)
Test = TensorDataset(X_test, y_test)

Train_loader = DataLoader(Train, batch_size=32, shuffle=True)
Test_loader = DataLoader(Test, batch_size=32, shuffle=True)

# Model Create

In [127]:
class CNN(nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv1d(1,32,kernel_size = 2),
            nn.ReLU(),
            nn.MaxPool1d(2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv1d(32,64, kernel_size =2),
            nn.ReLU(),
            nn.MaxPool1d(2)
        )
    
        self.fc = nn.Linear(4736,1, bias = True)
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        #print(out.shape)
        out = out.view(out.size(0),-1)
        #print(out.shape)
        out = self.fc(out)
        return torch.sigmoid(out)


# Model Train

In [128]:
# instantiate CNN model
model = CNN().to(device)

In [160]:
# parameters
learning_rate = 0.001
training_epochs = 100
batch_size = 32

In [161]:
# define cost/loss & optimizer
criterion = torch.nn.BCELoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [163]:
# train my model
total_train_batch = len(Train_loader)
total_test_batch = len(Test_loader)

print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0
    avg_acc = 0
    for X, Y in Train_loader:
        X = X.to(device)
        Y = Y.to(device)
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        avg_acc += ( ((model(X)).to(device)).int() == Y.to(device)).sum()
        avg_cost += cost / total_train_batch
    avg_acc =avg_acc/ 1153
    with torch.no_grad():
        avg_val_loss = 0
        val_loss = 0
        avg_val_acc = 0
        for X_val, Y_val in Test_loader:
            X_val = X_val.to(device)
            Y_val = Y_val.to(device)
            
            y_val_pred = model(X_val)
            val_step_loss = criterion(y_val_pred,Y_val)
            avg_val_loss += val_step_loss / total_test_batch
            avg_val_acc += ( ((model(X_val)).to(device)).int() == Y_val.to(device)).sum()
        avg_val_acc = avg_val_acc/ 289
    print('[Epoch: {:>4}] train_cost = {:>.9}, valid_cost = {:>.9}'.format(epoch + 1, avg_cost,avg_val_loss))
    print('[Epoch: {:>4}] train_acc = {:>.9}, valid_acc = {:>.9}'.format(epoch + 1, avg_acc, avg_val_acc))


print('Learning Finished!')

Learning started. It takes sometime.
[Epoch:    1] train_cost = 0.00640189555, valid_cost = 1.7665441
[Epoch:    1] train_acc = 0.909800529, valid_acc = 0.844290674
[Epoch:    2] train_cost = 0.00640338799, valid_cost = 1.36358964
[Epoch:    2] train_acc = 0.910667837, valid_acc = 0.844290674
[Epoch:    3] train_cost = 0.00640301127, valid_cost = 1.62802243
[Epoch:    3] train_acc = 0.907198608, valid_acc = 0.844290674
[Epoch:    4] train_cost = 0.00640277285, valid_cost = 1.36403084
[Epoch:    4] train_acc = 0.912402451, valid_acc = 0.844290674
[Epoch:    5] train_cost = 0.00640347693, valid_cost = 1.63103998
[Epoch:    5] train_acc = 0.908933222, valid_acc = 0.844290674
[Epoch:    6] train_cost = 0.00640286226, valid_cost = 1.36563969
[Epoch:    6] train_acc = 0.914137065, valid_acc = 0.847750902
[Epoch:    7] train_cost = 0.00640346855, valid_cost = 1.63023698
[Epoch:    7] train_acc = 0.913269758, valid_acc = 0.844290674
[Epoch:    8] train_cost = 0.00640481617, valid_cost = 1.6308

[Epoch:   65] train_acc = 0.925411999, valid_acc = 0.854671299
[Epoch:   66] train_cost = 0.00640235934, valid_cost = 1.91110468
[Epoch:   66] train_acc = 0.941023409, valid_acc = 0.854671299
[Epoch:   67] train_cost = 0.00640282268, valid_cost = 1.91139352
[Epoch:   67] train_acc = 0.944492638, valid_acc = 0.854671299
[Epoch:   68] train_cost = 0.00640405156, valid_cost = 1.91338289
[Epoch:   68] train_acc = 0.936686933, valid_acc = 0.854671299
[Epoch:   69] train_cost = 0.00640260288, valid_cost = 1.91250312
[Epoch:   69] train_acc = 0.946227252, valid_acc = 0.854671299
[Epoch:   70] train_cost = 0.00640744297, valid_cost = 1.91447163
[Epoch:   70] train_acc = 0.939288855, valid_acc = 0.854671299
[Epoch:   71] train_cost = 0.00640309695, valid_cost = 1.91158724
[Epoch:   71] train_acc = 0.952298343, valid_acc = 0.854671299
[Epoch:   72] train_cost = 0.00640219403, valid_cost = 1.91534352
[Epoch:   72] train_acc = 0.943625331, valid_acc = 0.858131528
[Epoch:   73] train_cost = 0.00640